In [ ]:
import requests
import csv
import time
from bs4 import BeautifulSoup
from collections import Counter
import praw
import pandas as pd
import datetime as dt #only if you want to analyze the date created feature
import json 

In [2]:
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set): return list(obj)
        return json.JSONEncoder.default(self, obj)

In [3]:
# Opening JSON file 
f = open('dont-change.json',) 
  
# returns JSON object as  
# a dictionary 
data = json.load(f)['selection2']
  

unique = [ each['name'].replace("u/", "") for each in data]

with open('result.json', 'w') as fp:
    json.dump(unique, fp)

    
f.close() 

In [4]:
reddit = praw.Reddit(client_id='A1ufXY3vdi2vpA', 
                     client_secret='nO1tFwJCFGiegR4NQzcMEbPze0g', 
                     user_agent='Script', 
                     username='', 
                     password='')

In [18]:
def get_dict_from_submission(submission):
    return {'sub': submission.subreddit.display_name, \
            'title': submission.title, \
            'body' : submission.selftext, \
            'num_u' : submission.score, \
            'url' : submission.url, \
            'num_c' : submission.num_comments, \
            'time': submission.created \
    }

def init_vars():
    return [], 0,0

def print_progress(user_count):
    if user_count%50==0:  print("Handling user: ", user_count)  

def process_user(user):
    global user_count
    print_progress(user_count)
    user_count += 1
    
def dump_data(file_name, data):
    if isinstance(data, set): 
        data = json.dumps(data, cls=SetEncoder)
        
    with open(file_name, 'w') as fp:
        json.dump(data, fp)    

In [19]:
# STATICS

TOTAL_USERS = 1000
LIM = 50
MENTAL_ILLNESS_SUBREDDITS = ['MentalHealthSupport','antidepressants','socialanxiety','medical_advice','raisedbynarcissists','GeneticCounseling','ADHDAccountability','Dissociation','Anxiety', 'AnxietyDepression', 'socialanxiety', 'panicdisorder', 'PanicParty', 'HealthAnxiety', 'depression', 'SuicideWatch', 'OCD' , 'TrueOffMyChest', 'mentalhealth', 'offmychest', 'emotionalsupport', 'therapy', 'ADHD','AskDocs']


# COUNTERS

user_count = 0
total_post_count = 0


# RESULTS

results = {}
non_mental_health_subs = set()
subreddit_counter = Counter()

# get the submissions

users_submissions = [(reddit.redditor(user).submissions.new(limit=LIM), user) for user in unique[0:TOTAL_USERS]]


# main processing happens here:

for submissions, user in users_submissions:
    
    process_user(user) # print and increment count
    
    user_result, mental_ill_count, control_count = init_vars() # initializes empty variables
  
    try: # because 404 HTTP errors in reading submissions sometimes
        for submission in submissions:
            
            try: # to avoid null etc. errors
                
                # if title doesn't exist 
                # or body doesn't exist
                # or we already have 10 posts from user
                # or post is removed
                # then skip
                
                if not submission.title \
                or not submission.selftext \
                or (mental_ill_count > 5 and control_count > 5) \
                or submission.selftext == "[removed]": 
                    continue 

                # sub-reddit related work 
                
                subreddit_counter[submission.subreddit.display_name] += 1
                
                if submission.subreddit.display_name in MENTAL_ILLNESS_SUBREDDITS: mental_ill_count += 1
                else: 
                    control_count += 1
                    non_mental_health_subs.add(submission.subreddit.display_name)

                    
                # add the post to user's results
                total_post_count += 1
                user_result.append(get_dict_from_submission(submission))
                
            except:
                continue
        
        # if the user has at least 2 posts in each group, add to final result
        if(mental_ill_count > 2 and control_count > 2): results[user] = (user_result)
            
    except:
        continue
    

Handling user:  0
Handling user:  50
Handling user:  100
Handling user:  150
Handling user:  200
Handling user:  250
Handling user:  300
Handling user:  350
Handling user:  400
Handling user:  450
Handling user:  500
Handling user:  550
Handling user:  600
Handling user:  650
Handling user:  700
Handling user:  750
Handling user:  800
Handling user:  850
Handling user:  900
Handling user:  950


In [20]:
dump_data('reddit-data.json', results)
dump_data('reddit_sub_names.json', non_mental_health_subs)
dump_data('reddit_sub_counter.json', subreddit_counter.most_common())

In [21]:
print(total_post_count)

11526
